# Lichess Games

In [120]:
import berserk, pandas, time
from datetime import datetime
from tqdm import tqdm

# Token file in marisa's folder
with open('lichess.token') as f:
    API_TOKEN = f.read()

session = berserk.TokenSession(API_TOKEN)
client = berserk.Client(session=session)

**Get top players**

In [100]:
top_users = list(client.users.get_all_top_10()) + ['LeelaChess']
top_users

['bullet',
 'blitz',
 'rapid',
 'classical',
 'ultraBullet',
 'crazyhouse',
 'chess960',
 'kingOfTheHill',
 'threeCheck',
 'antichess',
 'atomic',
 'horde',
 'racingKings',
 'LeelaChess']

**Show what game data looks like for a player**

In [121]:
start = berserk.utils.to_millis(datetime(2010, 1, 1))
end = berserk.utils.to_millis(datetime(2020, 1, 1))

list(client.games.export_by_player('LeelaChess', since=start, until=end, max=3))

[{'id': 'r3hMCWrp',
  'rated': False,
  'variant': 'standard',
  'speed': 'blitz',
  'perf': 'blitz',
  'createdAt': datetime.datetime(2020, 4, 14, 2, 57, 30, 645000, tzinfo=datetime.timezone.utc),
  'lastMoveAt': datetime.datetime(2020, 4, 14, 2, 58, 40, 355000, tzinfo=datetime.timezone.utc),
  'status': 'outoftime',
  'players': {'white': {'user': {'name': 'LeelaChess',
     'title': 'BOT',
     'id': 'leelachess'},
    'rating': 2658},
   'black': {'user': {'name': 'rabidllama', 'id': 'rabidllama'},
    'rating': 1538}},
  'winner': 'white',
  'moves': 'e4 e5 Nf3 Nc6 Bc4 h6 d4 exd4 Nxd4 Nxd4 Qxd4 d6 Nc3',
  'clock': {'initial': 300, 'increment': 0, 'totalTime': 300}},
 {'id': 'D6FXEI5C',
  'rated': False,
  'variant': 'standard',
  'speed': 'blitz',
  'perf': 'blitz',
  'createdAt': datetime.datetime(2020, 4, 14, 2, 55, 4, 952000, tzinfo=datetime.timezone.utc),
  'lastMoveAt': datetime.datetime(2020, 4, 14, 2, 57, 24, 99000, tzinfo=datetime.timezone.utc),
  'status': 'resign',
  'pl

**Get game data for all top users**

In [127]:
max_games = 100 # Limit games queried per user, if needed

games = []
for user in tqdm(top_users):
    games.append(list(client.games.export_by_player(user, since=start, until=end, max=max_games)))
    time.sleep(5) # Don't overload API


100%|██████████| 14/14 [01:27<00:00,  6.28s/it]


In [128]:
games_df = pd.concat([pd.json_normalize(game) for game in games], ignore_index=True)
games_df['points1'] = games_df.apply(lambda row: 1 if row.winner == "white" else 0, axis = 1)
games_df['points2'] = games_df.apply(lambda row: 1 if row.winner == "black" else 0, axis = 1)
games_df = games_df.rename(columns={
    "createdAt": "date_time",
    "players.white.user.name": "player1_name", 
    "players.black.user.name": "player2_name"
})[["date_time", "player1_name", "player2_name", "points1", "points2"]]
games_df.sort_values(by='date_time').reset_index(drop=True)

,date_time,player1_name,player2_name,points1,points2
0,2011-05-15 10:54:41+00:00,BULLET,NaN,0,1
1,2011-05-15 11:11:45+00:00,BULLET,NaN,0,1
2,2011-05-15 11:16:46+00:00,ngrju,BULLET,0,1
3,2011-05-15 11:26:28+00:00,NaN,BULLET,1,0
4,2011-05-15 11:28:34+00:00,BULLET,suvarnabhumi,0,1
...,...,...,...,...,...
485,2020-04-14 02:38:43.738000+00:00,LeelaChess,kopdog,1,0
486,2020-04-14 02:44:39.111000+00:00,LeelaChess,kopdog,1,0
487,2020-04-14 02:53:14.782000+00:00,LeelaChess,longboardacat,1,0
488,2020-04-14 02:55:04.952000+00:00,Limbert15,LeelaChess,0,1
